In [20]:
 import pandas as pd
from datetime import datetime


#prepare data before importing to database
chunksize = 100000
chunks = pd.read_csv("data/Google-Playstore.csv", chunksize=chunksize)

def convert_size(value):
    value = value.replace(",", "")

    if "G" in value:
        return float(value.replace("G", "")) * 1_000_000_000
    elif "M" in value:
        return float(value.replace("M", "")) * 1_000_000  # Convert M to numeric
    elif "k" in value:
        return float(value.replace("k", "")) * 1_000  # Convert K to numeric
    elif "Varies with device" in value:
        return None  # Handle special case
    else:
        return float(value)  # Already numeric

def convert_dates(date):
    if date != "nan":
        date_obj = datetime.strptime(date, "%b %d, %Y")
        return date_obj.strftime("%d-%m-%Y")
    else:
        return None


df_list = []
for chunk in chunks:
    bool_columns = ["Free", "Ad Supported", "In App Purchases", "Editors Choice"]
    chunk[bool_columns] = chunk[bool_columns].astype(bool).replace({True: 1, False: 0}).astype(str)
    chunk["Installs"] = chunk["Installs"].astype(str).replace(r"[+,]", "", regex=True).astype(str)
    chunk["Size"] = chunk["Size"].astype(str).apply(convert_size)
    chunk["Minimum Android"] = chunk["Minimum Android"].astype(str).replace(r"(W|and up)\b", "", regex=True).str.strip()
    chunk["Released"] = chunk["Released"].astype(str).apply(convert_dates)
    chunk["Last Updated"] = chunk["Last Updated"].astype(str).apply(convert_dates)
    df_list.append(chunk)

df = pd.concat(df_list, ignore_index=True)
df.drop("Scraped Time", axis=1, inplace=True)
df.to_csv("data/Google-Playstore_cleaned.csv", index=False)
df.head(50)

C:\Users\pawel\AppData\Local\Temp\ipykernel_4732\2118220262.py:34: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  chunk[bool_columns] = chunk[bool_columns].astype(bool).replace({True: 1, False: 0}).astype(str)
C:\Users\pawel\AppData\Local\Temp\ipykernel_4732\2118220262.py:34: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  chunk[bool_columns] = chunk[bool_columns].astype(bool).replace({True: 1, False: 0}).astype(str)
C:\Users\pawel\AppData\Local\Temp\ipykernel_4732\2118220262.py:34: FutureWarning: Downcasting behavior in `replace` is

,App Name,App Id,Category,Rating,Rating Count,Installs,Minimum Installs,Maximum Installs,Free,Price,...,Developer Id,Developer Website,Developer Email,Released,Last Updated,Content Rating,Privacy Policy,Ad Supported,In App Purchases,Editors Choice
0,Gakondo,com.ishakwe.gakondo,Adventure,0.0,0.0,10,10.0,15,1,0.0,...,Jean Confident Irénée NIYIZIBYOSE,https://beniyizibyose.tk/#/,jean21101999@gmail.com,26-02-2020,26-02-2020,Everyone,https://beniyizibyose.tk/projects/,0,0,0
1,Ampere Battery Info,com.webserveis.batteryinfo,Tools,4.4,64.0,5000,5000.0,7662,1,0.0,...,Webserveis,https://webserveis.netlify.app/,webserveis@gmail.com,21-05-2020,06-05-2021,Everyone,https://dev4phones.wordpress.com/licencia-de-uso/,1,0,0
2,Vibook,com.doantiepvien.crm,Productivity,0.0,0.0,50,50.0,58,1,0.0,...,Cabin Crew,NaN,vnacrewit@gmail.com,09-08-2019,19-08-2019,Everyone,https://www.vietnamairlines.com/vn/en/terms-an...,0,0,0
3,Smart City Trichy Public Service Vehicles 17UC...,cst.stJoseph.ug17ucs548,Communication,5.0,5.0,10,10.0,19,1,0.0,...,Climate Smart Tech2,http://www.climatesmarttech.com/,climatesmarttech2@gmail.com,10-09-2018,13-10-2018,Everyone,NaN,1,0,0
4,GROW.me,com.horodyski.grower,Tools,0.0,0.0,100,100.0,478,1,0.0,...,Rafal Milek-Horodyski,http://www.horodyski.com.pl,rmilekhorodyski@gmail.com,21-02-2020,12-11-2018,Everyone,http://www.horodyski.com.pl,0,0,0
5,IMOCCI,com.imocci,Social,0.0,0.0,50,50.0,89,1,0.0,...,Imocci GmbH,http://www.imocci.com,info@imocci.com,24-12-2018,20-12-2019,Teen,https://www.imocci.com/wp-content/uploads/2018...,0,1,0
6,unlimited 4G data prank free app,getfreedata.superfatiza.unlimitedjiodataprank,Libraries & Demo,4.5,12.0,1000,1000.0,2567,1,0.0,...,android developer779,NaN,aitomgharfatimezzahra@gmail.com,23-09-2019,27-09-2019,Everyone,https://sites.google.com/view/unlimited4gdatap...,1,0,0
7,The Everyday Calendar,com.mozaix.simoneboard,Lifestyle,2.0,39.0,500,500.0,702,1,0.0,...,Mozaix LLC,NaN,elementuser03@gmail.com,21-06-2019,21-06-2019,Everyone,https://www.freeprivacypolicy.com/privacy/view...,0,0,0
8,WhatsOpen,com.whatsopen.app,Communication,0.0,0.0,10,10.0,18,1,0.0,...,Yilver Molina Hurtatiz,http://yilvermolinah.com,yilver.mh1996@gmail.com,None,07-12-2018,Teen,http://elcafedelamanana.yilvermolinah.com/poli...,0,0,0
9,Neon 3d Iron Tech Keyboard Theme,com.ikeyboard.theme.neon_3d.iron.tech,Personalization,4.7,820.0,50000,50000.0,62433,1,0.0,...,Free 2021 Themes for Emoji keyboard,https://trendyteme888-31139.web.app,trendyteme.888@gmail.com,22-09-2019,07-10-2020,Everyone,http://bit.ly/EmojiThemePro,1,0,0
